In [1]:
# ArrayList and Stack code to use

class Stack():
    def __init__(self):
        self.inList = None
        self.size = 0
        
    def push(self, v):
        self.size += 1
        self.inList = (v,self.inList)
        
    def pop(self):
        if self.size == 0: assert(0)
        self.size -= 1
        (v,ls) = self.inList
        self.inList = ls
        return v
    
    def __str__(self):
        s = "["
        ls = self.inList
        for _ in range(self.size):
            (v,ls) = ls
            s += str(v)
            if ls!=None: s += ", "
        return s+"]"
        
class ArrayList:
    def __init__(self):
        self.inArray = [0 for i in range(10)]
        self.count = 0
        
    def get(self, i):
        return self.inArray[i]

    def set(self, i, e):
        self.inArray[i] = e

    def length(self):
        return self.count

    def append(self, e):
        self.inArray[self.count] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()

    def insert(self, i, e):
        for j in range(self.count,i,-1):
            self.inArray[j] = self.inArray[j-1]
        self.inArray[i] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()
    
    def remove(self, i):
        self.count -= 1
        val = self.inArray[i]
        for j in range(i,self.count):
            self.inArray[j] = self.inArray[j+1]
        return val

    def _resizeUp(self):
        newArray = [0 for i in range(2*len(self.inArray))]
        for j in range(len(self.inArray)):
            newArray[j] = self.inArray[j]
        self.inArray = newArray
        
    def toArray(self):
        return self.inArray[:self.count]

    def __str__(self):
        if self.count == 0: return "[]"
        s = "["
        for i in range(self.count-1): s += str(self.inArray[i])+", "
        return s+str(self.inArray[self.count-1])+"]" 



class ArrayListWithUndo(ArrayList):
    def __init__(self):
        super().__init__()
        self.undos = Stack()

    def set(self, i, v):
        # already implemented
        self.undos.push(("set",i,self.inArray[i]))
        self.inArray[i] = v

    def append(self, v):
        self.undos.push(("rem", self.count, None))
        super().append(v)

    def insert(self, i, v):
        self.undos.push(("rem", i, None))
        super().insert(i, v)

    def remove(self, i):
        self.undos.push(("ins", i, self.get(i)))
        super().remove(i)

    def undo(self):
        if self.undos.size == 0:
            return
    
        recent = self.undos.pop()
        action, index, value = recent
    
        match action:
            case "set":
                super().set(index, value)
            case "rem":
                super().remove(index)
            case "ins":
                super().insert(index, value)
            case _:
                print("Error: Unknown action", action)

    def __str__(self):
        return str(self.toArray()) + "\n-> " + str(self.undos)



class NetworkWithUndo:
    def __init__(self, N):
        # already implemented
        self.inArray = ArrayListWithUndo()
        for _ in range(N): self.inArray.append(-1)
        self.undos = Stack()
        self.undos.push(N)

    def getSize(self):
         # already implemented
         return self.inArray.length()
       
    def add(self):
        #TODO
        node_index = self.inArray.length()
        if node_index != -1: 
            self.inArray.append(-1)
            self.undos.push(1)

    def root(self, i):
        #TODO
        stack = Stack()
        node_pointer = i
        steps = 0  

        while self.inArray.get(node_pointer) >= 0:
            stack.push(node_pointer)
            node_pointer = self.inArray.get(node_pointer)
            steps += 1  

        root = node_pointer

        path = 0
        while stack.size > 0:
            node = stack.pop()
            parent_node = self.inArray.get(node)
            if parent_node != root:
                self.inArray.set(node, root)
                path += 1

        if path > 0:
            self.undos.push(path)

        return root

    def merge(self, i, j):
        #TODO
        node1 = self.root(i)
        node2 = self.root(j)

        if node1 == node2:
            return

        node1_total = -self.inArray.get(node1)
        node2_total = -self.inArray.get(node2)

        if node1_total == node2_total:

            self.inArray.set(node1, node2)
            self.inArray.set(node2, -(node1_total + node2_total))
        else:
            if node1_total > node2_total:
  
                self.inArray.set(node2, node1)
                self.inArray.set(node1, -(node1_total + node2_total))
            else:
                self.inArray.set(node1, node2)
                self.inArray.set(node2, -(node1_total + node2_total))

        self.undos.push(2)

    def undo(self):
        #TODO
        if self.undos.size == 0:
            return

        counter = 0
        n = self.undos.pop()
        i = 0
        while i < n:
            self.inArray.undo()   
            counter += 1
            i += 1                

    def toArray(self):
        # already implemented
        return self.inArray.toArray()

    def __str__(self):
        # already implemented
        return str(self.toArray())+"\n-> "+str(self.undos)


class Gadget:
    def __init__(self):
        # already implemented
        self.inNetwork = NetworkWithUndo(0)
        self.subsize = 0
        self.nameMap = {} #dictionary used to find node number from name
        self.undos = Stack()
        self.helper = {} #dictionary used to find name from node number
        
    def getSize(self):
        # already implemented
        return self.inNetwork.getSize()
        
    def isIn(self, name):
        # already implemented
        return name in self.nameMap
        
    def add(self, name):
        if self.isIn(name):
            self.undos.push(("oth",0,None))
            return
        self.nameMap[name] = self.getSize()
        self.helper[self.getSize()] = name
        self.inNetwork.add()
        self.subsize += 1
        self.undos.push(("rem",1,name))
    
    def connect(self, name1, name2):
        root1 = self.inNetwork.root(self.nameMap[name1])
        root2 = self.inNetwork.root(self.nameMap[name2])
        if root1 == root2:
            return True
        self.inNetwork.merge(root1, root2)
        self.subsize -= 1
        self.undos.push(("brk",1,None))
        return False
        
    def clean(self, name):
        while name in self.nameMap:
            self.undo(1)
            if self.getSize() == 0:
                return

    def subnets(self):
        subnets = {}
        for i in range(self.getSize()):
            root = self.inNetwork.root(i)
            if root in subnets:
                tempArray = subnets[root]
            else:
                tempArray = ArrayList()
            i = self.helper[i]
            tempArray.append(i)
            subnets[root] = tempArray
        for i in subnets:
            subnets[i] = subnets[i].toArray()
        return self.dictToArray(subnets)
                    
    def removeFromDict(self,name):
        newNameDict = {}
        newHelperDict = {}
        for i in self.nameMap:
            if i != name:
                newNameDict[i] = self.nameMap[i]
                newHelperDict[self.nameMap[i]] = i
        self.nameMap = newNameDict
        self.helper = newHelperDict
    
    def undo(self, n):
        for i in range (n,0,-1):
            if self.getSize() == 0:
                return
            currentUndo = self.undos.pop()
            op, num, s = currentUndo
            if op == "rem":
                self.removeFromDict(s) #this function replaces pop function for dictionary as it is built in function and so cannot be used
                self.subsize -= 1
            elif op == "brk":
                    self.subsize += 1
            for j in range (num):
                self.inNetwork.undo()
        
    def toArray(self):
        # already implemented
        A = self.inNetwork.toArray()
        for s in self.nameMap:
            i = self.nameMap[s]
            A[i] = (s,A[i])
        return A

    def dictToArray(self,dictArray):
        newArray = ArrayList()
        for i in dictArray:
            newArray.append(dictArray[i])
        return newArray.toArray()
        
    def __str__(self):
        # already implemented
        return str(self.toArray())+"\n-> "+str(self.nameMap)+"\n-> "+str(self.undos)
